In [1]:
# Install dependencies
pip install -U datasets transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.4/41.4 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 506.3/506.3 kB 11.7 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 86.5 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 564.3/564.3 kB 37.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 42.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 85.9 MB/s eta 0:00:00:00:01
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 19.0.1
    Uninstalling pyarrow-19.0.1:
      Successfully uninstalled pyarrow-19.0.1
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 1.0.0rc2
    Uninstalling huggingface-hub-1.0.0rc2:
      Successfully uninstalled huggingface-hub-1.0.0rc2
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.21.2
    Uninstalling tokenizers-0.21.2:
      Success

In [2]:
# Problem 5 Prompt Engineering
# Part 1: Dataset Setup
from datasets import load_dataset

dataset = load_dataset("openai/gsm8k", "main")
dataset = dataset.shuffle(seed=42)

#Splitting the dataset into train and test sets
train_dataset = dataset["train"]
test_dataset = dataset["test"]

# Randomly selecting first example from shuffled test dataset for my experiments.
print(test_dataset[0]['question'])
print(test_dataset[0]['answer'])

README.md: 0.00B [00:00, ?B/s]

main/train-00000-of-00001.parquet:   0%|          | 0.00/2.31M [00:00<?, ?B/s]

main/test-00000-of-00001.parquet:   0%|          | 0.00/419k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/7473 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1319 [00:00<?, ? examples/s]

Darrell and Allen's ages are in the ratio of 7:11. If their total age now is 162, calculate Allen's age 10 years from now.
The total ratio representing their ages is 7+11= <<7+11=18>>18
Since the fraction of the ratio that represents Allen's age is 11/18, Allen's current age is 11/18*162 = <<11/18*162=99>>99
If Allen is currently 99 years old, in 10 years he will be 99+10 = <<99+10=109>>109 years old
#### 109


In [8]:
# Part 2: Model Selection
# Choosing microsoft/Phi-4-mini-reasoning LLM from the Hugging Face Model Hub
# Best quality LLM that's small enough to run fast on Kaggle GPU
model_id = "microsoft/Phi-4-mini-reasoning" 

In [9]:
# Printing initial examples I'm using in prompting for reference
print(test_dataset[1]['question'])
print()
print(test_dataset[1]['answer'])
print()
print()
print(test_dataset[2]['question'])
print()
print(test_dataset[2]['answer'])

Lorraine and Colleen are trading stickers for buttons. Each large sticker is worth a large button or three small buttons. A small sticker is worth one small button. A large button is worth three small stickers. Lorraine starts with 30 small stickers and 40 large stickers. She trades 90% of her small stickers for large buttons. She trades 50% of her large stickers for large buttons and trades the rest of them for small buttons. How many buttons does she have by the end?

She trades 27 small stickers because 30 x .9 = <<27=27>>27
She gets 9 large buttons for these because 27 / 3 = <<27/3=9>>9
She trades 20 large stickers for large buttons because 40 x .5 = 20
She gets 20 large buttons for these because 20 / 1 = <<20/1=20>>20
She trades 50% of her large stickers for small buttons because 100 - 50 = <<100-50=50>>50
She trades 20 large stickers for small buttons because 40 x .5 = 20
She gets 60 small buttons because 20 x 3 = <<20*3=60>>60
She has 89 buttons at the end because 9 + 20 + 60 = 

In [21]:
# Part 3: Prompt Engineering
# All you need to do is run the cell to see the generated output for each prompting technique
from transformers import pipeline
generator = pipeline(
    "text-generation",
    model=model_id,
    device_map="auto"
)

def generate_solution(prompt, problem):
    messages = [
        {"role": "system", "content": prompt},
        {"role": "user", "content": problem}
    ]
    response = generator(messages, max_new_tokens=512)
    return response[0]["generated_text"]

def one_shot_prompting_numeric(prompt, problem):
    example_input = test_dataset[1]['question']
    example_output = "89"
    full_user_prompt = f"""
    Example:
    Q: {example_input}
    A: {example_output}
    
    Now solve this new problem:
    Q: {problem}
    A:
    """
    messages = [
        {"role": "system", "content": prompt},
        {"role": "user", "content": full_user_prompt}
    ]
    response = generator(messages, max_new_tokens=512)
    return response[0]["generated_text"]

def two_shot_prompting_numeric(prompt, problem):
    example_input = test_dataset[1]['question']
    example_output = "89"

    example_input_2 = test_dataset[2]['question']
    example_output_2 = "13"
    
    full_user_prompt = f"""
    Example:
    Q: {example_input}
    A: {example_output}

    Q: {example_input_2}
    A: {example_output_2}
    
    Now solve this new problem:
    Q: {problem}
    A:
    """
    messages = [
        {"role": "system", "content": prompt},
        {"role": "user", "content": full_user_prompt}
    ]
    response = generator(messages, max_new_tokens=512)
    return response[0]["generated_text"]

def two_shot_cot_prompting_numeric(prompt, problem):
    example_input = test_dataset[1]['question']
    example_output = test_dataset[1]['answer']

    example_input_2 = test_dataset[2]['question']
    example_output_2 = test_dataset[2]['answer']
    
    full_user_prompt = f"""
    Example:
    Q: {example_input}
    A: {example_output}

    Q: {example_input_2}
    A: {example_output_2}
    
    Now solve this new problem:
    Q: {problem}
    A:
    """
    messages = [
        {"role": "system", "content": prompt},
        {"role": "user", "content": full_user_prompt}
    ]
    response = generator(messages, max_new_tokens=512)
    return response[0]["generated_text"]

system_prompt = "You are a precise and reliable math problem solving assistant."
system_prompt_cot = "You are a careful math expert. Show full step-by-step reasoning (chain of thought) while solving problems. Do not abbreviate steps."

print(generate_solution(system_prompt, test_dataset[0]['question']))
print()
print(one_shot_prompting_numeric(system_prompt, test_dataset[0]['question']))
print()
print(two_shot_prompting_numeric(system_prompt, test_dataset[0]['question']))
print()
print(two_shot_cot_prompting_numeric(system_prompt_cot, test_dataset[0]['question']))

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cuda:0


[{'role': 'system', 'content': 'You are a precise and reliable math problem solving assistant.'}, {'role': 'user', 'content': "Darrell and Allen's ages are in the ratio of 7:11. If their total age now is 162, calculate Allen's age 10 years from now."}, {'role': 'assistant', 'content': "<think>\nOkay, let me try to figure out this problem. So, the problem says that Darrell and Allen's ages are in the ratio 7:11, and their total age now is 162. We need to find Allen's age 10 years from now. Hmm, ratios can sometimes be tricky, but I think I can handle this step by step.\n\nFirst, let me recall that when ages are given in a ratio, it means that for every 7 units of Darrell's age, Allen has 11 units. So, their ages can be represented as multiples of these numbers. Let me denote Darrell's age as 7x and Allen's age as 11x, where x is a common factor. That makes sense because 7x:11x is the same as 7:11 when simplified.\n\nNow, the problem states that their total age is 162. So, if I add Darre

In [22]:
# Printing new examples I'm using in prompting for reference
import random 
random.seed(42)

# getting random index to use to access an example in test_dataset
random_integer = random.randint(0, 1318)
print("Index: " + str(random_integer))
print(test_dataset[random_integer]['question'])
print()
print(test_dataset[random_integer]['answer'])

print()
print()

# getting random index to use to access an example in test_dataset
random_integer = random.randint(0, 1318)
print("Index: " + str(random_integer))
print(test_dataset[random_integer]['question'])
print()
print(test_dataset[random_integer]['answer'])

Index: 1309
Sean is practicing for his role in a theater production. He has to memorize his lines for two scenes and the lyrics to one solo song. His solo song has 54 lines in the lyrics. The first scene has twice the number of lines, but only a third of them are his lines. The second scene has six more lines than the song, and four-fifths of them are his. How many lines does Sean have to memorize?

The first scene has twice the number of lines as his solo song, so it has 2 * 54 = <<2*54=108>>108 lines.
A third of them are Sean’s lines, so he has to memorize 108 / 3 = <<108/3=36>>36 lines.
The second scene has 54 + 6 = <<54+6=60>>60 lines.
Four fifths of the second scene’s lines are Sean’s, so he has to memorize 4 * 60 / 5 = 4 * 12 = <<4*60/5=48>>48 lines.
Sean has to memorize his entire solo song, so he has to memorize 54 + 36 + 48 = <<54+36+48=138>>138 lines in all.
#### 138


Index: 228
Mitchell is making nachos for his family. He buys two bags of chips with 55 chips each. If his fa

In [23]:
# Part 4: Prompt Refinement
# I changed the phrasing of the user prompts to be more specific to how the output should be.
# I changed the examples provided in the context to 2 other examples in the test set
# I changed the system prompt to be more specific to the task at hand
# I changed the system chain of thought prompt to be more brief and concise

# All you need to do is run the cell to see the generated output for each prompting technique

generator = pipeline(
    "text-generation",
    model=model_id,
    device_map="auto"
)
example_input = test_dataset[1309]['question']
example_output_ans = "138"
example_output = test_dataset[1309]['answer']

example_input_2 = test_dataset[228]['question']
example_output_2_ans = "22"
example_output_2 = test_dataset[228]['answer']

def generate_solution(prompt, problem):
    messages = [
        {"role": "system", "content": prompt},
        {"role": "user", "content": problem}
    ]
    response = generator(messages, max_new_tokens=512)
    return response[0]["generated_text"]

def one_shot_prompting_numeric(prompt, problem):
    full_user_prompt = f"""
    Example:
    Question: {example_input}
    Answer: {example_output_ans}
    
    Now derive the answer for this new problem and present the answer in the same format as presented in the example:
    Question: {problem}
    Answer:
    """
    messages = [
        {"role": "system", "content": prompt},
        {"role": "user", "content": full_user_prompt}
    ]
    response = generator(messages, max_new_tokens=512)
    return response[0]["generated_text"]

def two_shot_prompting_numeric(prompt, problem):
    full_user_prompt = f"""
    Example:
    Question: {example_input}
    Answer: {example_output_ans}

    Question: {example_input_2}
    Answer: {example_output_2_ans}
    
    Now derive the answer for this new problem and present the answer in the same format as presented in the examples:
    Question: {problem}
    Answer:
    """
    messages = [
        {"role": "system", "content": prompt},
        {"role": "user", "content": full_user_prompt}
    ]
    response = generator(messages, max_new_tokens=512)
    return response[0]["generated_text"]

def two_shot_cot_prompting_numeric(prompt, problem):
    full_user_prompt = f"""
    Example:
    Question: {example_input}
    Answer: {example_output}

    Question: {example_input_2}
    Answer: {example_output_2}
    
    Now derive the answer for this new problem and present the answer in the same format as presented in the examples:
    Question: {problem}
    Answer:
    """
    messages = [
        {"role": "system", "content": prompt},
        {"role": "user", "content": full_user_prompt}
    ]
    response = generator(messages, max_new_tokens=512)
    return response[0]["generated_text"]

system_prompt = "You are an algebra word problem expert that can accurately compute numerical answers."
system_prompt_cot = "You are an expert math tutor who breaks down complex word problems into smaller steps leading to an accurate solution."

# Part 5: Evaluation 
print(generate_solution(system_prompt, test_dataset[0]['question']))
print()
print(one_shot_prompting_numeric(system_prompt, test_dataset[0]['question']))
print()
print(two_shot_prompting_numeric(system_prompt, test_dataset[0]['question']))
print()
print(two_shot_cot_prompting_numeric(system_prompt_cot, test_dataset[0]['question']))

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cuda:1


[{'role': 'system', 'content': 'You are an algebra word problem expert that can accurately compute numerical answers.'}, {'role': 'user', 'content': "Darrell and Allen's ages are in the ratio of 7:11. If their total age now is 162, calculate Allen's age 10 years from now."}, {'role': 'assistant', 'content': "<think>\nOkay, let's see. The problem is about Darrell and Allen's ages. The ratio of their ages is 7:11, and together their total age is 162. I need to find Allen's age 10 years from now. Hmm, ratios can sometimes be tricky, but let me break it down step by step.\n\nFirst, ratios represent parts of a whole. So, if Darrell and Allen's ages are in the ratio 7:11, that means for every 7 years of Darrell, Allen has 11 years. The total number of parts would be 7 + 11. Let me check that: 7 plus 11 is 18. So, the total ratio parts are 18.\n\nNow, their combined age is 162. So, each part of the ratio must equal a certain number of years. To find out how much each part is worth, I can divi

In [24]:
# Part 5: Evaluation
"""
Even though most of the prompting techniques displayed good performance and resulted in the correct answer, there were some noticeable differences in their provided reasoning and derivation for the solution. The general prompting technique was really close to getting the correct answer but was unfortunately cut off by the max_new_token limit. For some reason the general prompting technique was hallucinating about how unrealistic Allen's age is which shouldn't matter in the context of a basic word problem. This hallucination may be due to the cause of not having received previous examples which might have helped the LLM to understand that the numbers in the word problems are not meant to be taken seriously. The rest of the prompting techniques were able to derive the solution correctly with all the major derivation steps shown in the output. Another thing I noticed was that all prompting techniques apart from general prompting double checked their answers in some way which was interesting since I didn't explicitly ask for this behavior in the prompt. There isn't much difference between the outputs for 1 shot prompting and two shot prompting, but the LLM has a tone that indicates its more confident in its solution for 2 shot prompting. Finally, the last major takeaway I had was how although the chain of thought prompting technique caused the LLM to effectively break the problem down into steps, it ended up breaking the problem down into too many elementary steps of which some I thought were a little unnecessary.
"""

"\nEven though most of the prompting techniques displayed good performance and resulted in the correct answer, there were some noticeable differences in their provided reasoning and derivation for the solution. The general prompting technique was really close to getting the correct answer but was unfortunately cut off by the max_new_token limit. For some reason the general prompting technique was hallucinating about how unrealistic Allen's age is which shouldn't matter in the context of a basic word problem. This hallucination may be due to the cause of not having received previous examples which might have helped the LLM to understand that the numbers in the word problems are not meant to be taken seriously. The rest of the prompting techniques were able to derive the solution correctly with all the major derivation steps shown in the output. Another thing I noticed was that all prompting techniques apart from general prompting double checked their answers in some way which was intere

In [25]:
# Part 6: Evaluation
"""
My prompt engineering experience was pretty smooth and interesting for the most part. It’s kind of difficult to state what was the most effective prompting strategies since all the prompting strategies apart from general prompting performed really well. I guess the LLM I chose might have been pre-trained or fine-tuned on a lot of basic algebra word problems which could explain the stellar performance. The most interesting thing I discovered was that although the chain of thought prompting technique effectively made the LLM break the problem down into steps, it ended up breaking the problem down into too many elementary steps of which some I thought were definitely unnecessary to mention. I like how the outputs for the 1-shot prompting and 2-shot prompting techniques were  more brief and concise in the derivation of the final solution. It was also really interesting how the general prompting technique failed to report an answer due to hallucinating about how unrealistic Allen's age is. This hallucination was probably due to not having context of other examples which would have helped the LLM to understand that the numbers in the word problems are not meant to be taken seriously.
"""

"\nMy prompt engineering experience was pretty smooth and interesting for the most part. It’s kind of difficult to state what was the most effective prompting strategies since all the prompting strategies apart from general prompting performed really well. I guess the LLM I chose might have been pre-trained or fine-tuned on a lot of basic algebra word problems which could explain the stellar performance. The most interesting thing I discovered was that although the chain of thought prompting technique effectively made the LLM break the problem down into steps, it ended up breaking the problem down into too many elementary steps of which some I thought were definitely unnecessary to mention. I like how the outputs for the 1-shot prompting and 2-shot prompting techniques were  more brief and concise in the derivation of the final solution. It was also really interesting how the general prompting technique failed to report an answer due to hallucinating about how unrealistic Allen's age i